In [1]:
import pandas as pd
import numpy as np

from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import pdb

In [2]:
text_reference_sampled_df = pd.read_csv('text_reference_sampled_df.csv')
text_reference_sampled_df.head()

,reference_id,reference_property_id,reference_datatype,url,netloc,netloc_agg,error_msg,code,content-type,final_url,reason,language_crawl,language_crawl_score,sampling_weight_vb,sampling_weight
0,390d6c6e68a32e11f8d7b0883cda0557db529fe6,P4656,url,https://en.wikipedia.org/w/index.php?title=G._...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=G._...,OK,en,0.934613,321744|13,24749.538462
1,49b9aec8e10815611ff0379a34d5fd7c3830566e,P4656,url,https://en.wikipedia.org/w/index.php?title=Seb...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Seb...,OK,en,0.650086,321744|13,24749.538462
2,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,P4656,url,https://en.wikipedia.org/w/index.php?title=Ave...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Ave...,OK,en,0.842498,321744|13,24749.538462
3,76b04346ad57869d9e5ae1007ba8343d708ab6f9,P4656,url,https://en.wikipedia.org/w/index.php?title=Yut...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Yut...,OK,en,0.608352,321744|13,24749.538462
4,1ee71b39caf6df395c64b436fea4895692812d38,P4656,url,https://en.wikipedia.org/w/index.php?title=Dol...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Dol...,OK,en,0.856581,321744|13,24749.538462


# Extract HTML

For each url in the sampled dataset, we will:
1. Visit the URL with a *headed* browser
2. Wait for the correct content to be rendered
3. Extract the html code and save it into a folder, but also in the dataframe

In [11]:
_RE_COMBINE_WHITESPACE = re.compile(r"\s+")

options = Options()
options.add_argument("--headless")
options.add_argument("--incognito")
#options.add_argument("start-maximized")
#options.add_argument("--disable-extensions")
#options.add_argument("--enable-javascript")
#options.add_argument("--window-size=1920x1080")
#options.add_experimental_option("excludeSwitches", ["enable-automation"])
#options.add_experimental_option('useAutomationExtension', False)
#options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36")

DRIVER_PATH = '/home/gabriela/selenium_drivers/chromedriver'
ser = Service(DRIVER_PATH)

text_reference_sampled_df['html'] = None

In [12]:
from os.path import exists
import os
from pathlib import Path

# Driver outside so we keep a single browser and history
# NOTE: Some URLs lead to PDF linked by 'embed' tags. Pursue those and try processing them with an adequate tools.
with webdriver.Chrome(options=options, service=ser) as driver:
    
    driver.set_window_position(-1500, 0)
    driver.maximize_window()
    for i, row in text_reference_sampled_df.iterrows():
        
        if not exists(Path('reference_html') / row.netloc):
            os.makedirs(Path('reference_html') / row.netloc)
        if exists(Path('reference_html') / row.netloc / row.reference_id):
            if text_reference_sampled_df.loc[i,'html'] is None:
                print(i,' - Loading from disk - ', row.url)
                with open(Path('reference_html') / row.netloc / row.reference_id, 'r', encoding='utf-8') as f:
                    text_reference_sampled_df.loc[i,'html'] = f.read()
            continue      
            
        print(i, row.url)
        driver.get(row.url)
        pdb.set_trace()
        html = driver.page_source
        
        with open(Path('reference_html') / row.netloc / row.reference_id, 'w+', encoding='utf-8') as f:
            f.write(html)
            
        text_reference_sampled_df.loc[i,'html'] = html
        
        clear_output(wait=True)
clear_output(wait=True)

0  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=G._V._Raja&oldid=916789538
1  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Sebastian_Sabol&oldid=921422667
2  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Average_White_Band&oldid=921360877
3  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Yutaka_Higuchi_(musician)&oldid=852886961
4  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Dolores_Delirio&oldid=846145679
5  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Josip_Bilinovac&oldid=919882149
6  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Alabama_State_Route_145&oldid=753973024
7  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Twin_Pimples_raid&oldid=912800699
8  - Loading from disk -  https://en.wikipedia.org/w/index.php?title=Ponniyin_Selvan_(2005_film)&oldid=982538956
9  - Loading from disk -  https://en.wikipedia.org/w/index.ph

181  - Loading from disk -  http://www.tate.org.uk/art/artworks/fontana-spatial-concept-t03961
182  - Loading from disk -  https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500005763
183  - Loading from disk -  https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500012917
184  - Loading from disk -  https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500021971
185  - Loading from disk -  https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500033472
186  - Loading from disk -  https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500330508
187  - Loading from disk -  https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500116902
188  - Loading from disk -  http://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500226683
189  - Loading from disk -  https://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500117440
190  - Loading from disk -

347  - Loading from disk -  https://www.allmusic.com/album/mw0000098473
348  - Loading from disk -  https://www.allmusic.com/album/mw0000601192
349  - Loading from disk -  https://www.allmusic.com/album/mw0000673051
350  - Loading from disk -  https://www.allmusic.com/album/mw0000074746
351  - Loading from disk -  https://www.historyofparliamentonline.org/volume/1509-1558/member/lacy-henry-1491-156465
352  - Loading from disk -  https://www.historyofparliamentonline.org/volume/1558-1603/member/cholmley-sir-roger-1495-1565
353  - Loading from disk -  https://www.historyofparliamentonline.org/volume/1386-1421/member/chaunceys-john
354  - Loading from disk -  https://www.historyofparliamentonline.org/volume/1386-1421/member/parker-william-ii-1421
355  - Loading from disk -  https://www.historyofparliamentonline.org/volume/1558-1603/member/manners-john-1535-1611
356  - Loading from disk -  https://www.historyofparliamentonline.org/volume/1509-1558/member/thornton-henry-1484-1533
357  - Loa

441  - Loading from disk -  http://witches.shca.ed.ac.uk/index.cfm?fuseaction=home.caserecord&caseref=C%2FEGD%2F2195&search_type=searchtrial&search_string=
442  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?19637
443  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?208733
444  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?219328
445  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?133274
446  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?26318
447  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?141128
448  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?110671
449  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?9791
450  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?168845
451  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?19931
452  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?5517
453  - Loading from disk -  http://www.isfdb.org/cgi-bin/ea.cgi?168217

602  - Loading from disk -  https://www.npg.org.uk/collections/search/person/mp55783
603  - Loading from disk -  https://www.npg.org.uk/collections/search/person/mp71404
604  - Loading from disk -  https://www.npg.org.uk/collections/search/person/mp75249
605  - Loading from disk -  https://www.npg.org.uk/collections/search/use-this-image.php?mkey=mw02734
606  - Loading from disk -  https://www.npg.org.uk/collections/search/person/mp53551
607  - Loading from disk -  http://www.npg.org.uk/collections/search/portrait/mw07912/Henry-Wickham-Steed
608  - Loading from disk -  http://www.npg.org.uk/collections/search/portrait/mw04976/Philip-II-King-of-Spain
609  - Loading from disk -  http://www.npg.org.uk/collections/search/portrait/mw00808/Rupert-Brooke
610  - Loading from disk -  https://www.npg.org.uk/collections/search/person/mp57012/stella-katherine-percy-nee-drummond-lady-percy-of-newcastle
611  - Loading from disk -  http://www.biographi.ca/en/bio/baile_joseph_alexandre_11E.html
612  -

``p driver.switch_to.frame('ext-gen66')
p BeautifulSoup(driver.page_source, "lxml").get_text(' ')``

**NOTE**: Some URLs lead to PDF linked by 'embed' tags. Pursue those and try processing them with an adequate tools.
    
    - https://www.bailii.org/(.*).html change html to pdf and process that instead if 'This item of legislation is only available to download and view as PDF' included in the html body.
    - https://www.eib.org/en/ also links to lots of pdfs
    
**NOTE**: Some content is not in English, like the following, which should be either removed or replaced.

    - https://www.archinform.net/projekte/19632.htm (290)
    - https://www.archinform.net/projekte/11996.htm (295)
    
Since it's 4 out of 676, let's just erase them.

In [25]:
#html = text_reference_sampled_df.html[42]
#
#
#soup = BeautifulSoup(html, "lxml")
#[s.decompose() for s in soup("script")]  # remove <script> elements
#body_text = _RE_COMBINE_WHITESPACE.sub(" ", soup.body.get_text(' ')).strip()
#body_text
#

In [13]:
text_reference_sampled_df

,reference_id,reference_property_id,reference_datatype,url,netloc,netloc_agg,error_msg,code,content-type,final_url,reason,language_crawl,language_crawl_score,sampling_weight_vb,sampling_weight,html
0,390d6c6e68a32e11f8d7b0883cda0557db529fe6,P4656,url,https://en.wikipedia.org/w/index.php?title=G._...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=G._...,OK,en,0.934613,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
1,49b9aec8e10815611ff0379a34d5fd7c3830566e,P4656,url,https://en.wikipedia.org/w/index.php?title=Seb...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Seb...,OK,en,0.650086,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
2,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,P4656,url,https://en.wikipedia.org/w/index.php?title=Ave...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Ave...,OK,en,0.842498,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
3,76b04346ad57869d9e5ae1007ba8343d708ab6f9,P4656,url,https://en.wikipedia.org/w/index.php?title=Yut...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Yut...,OK,en,0.608352,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
4,1ee71b39caf6df395c64b436fea4895692812d38,P4656,url,https://en.wikipedia.org/w/index.php?title=Dol...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Dol...,OK,en,0.856581,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,2f1cadf00c6698a24d7e2954d4e41abfab52b1f4,P854,url,https://indiancine.ma/AKJ,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/AKJ,OK,en,0.617740,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."
672,a22468e40d626f8347c1fcf49587332f4cbe1911,P854,url,https://indiancine.ma/COK,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/COK,OK,en,0.511245,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."
673,ddebe4072a35e892ac423dc9837b8f3a5e022766,P854,url,https://indiancine.ma/SJ,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/SJ,OK,en,0.543558,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."
674,78eb8ba32e77a9434161dabc2dee3b4dede3fa0b,P854,url,https://indiancine.ma/BCB,indiancine.ma,indiancine.ma,none,200,text/html; charset=utf-8,https://indiancine.ma/BCB,OK,en,0.612868,993|13,76.384615,"<html><head>\n <meta charset=""utf-8"">\n..."


In [ ]:
text_reference_sampled_df.to_csv('text_reference_sampled_df_html.csv', index=None)